<!-- ### REF https://pytorch.ac.cn/tutorials/intermediate/char_rnn_classification_tutorial.html -->

In [1]:
# import torch
# from torch import nn
# import torch.nn.functional as F
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.set_default_device(device=device)
# torch.get_default_device()

In [2]:
# import string # REF https://docs.python.org/zh-cn/3.11/library/string.html
# import unicodedata # REF https://docs.python.org/zh-cn/3.11/library/unicodedata.html

<!-- #### 1. 数据清理

将Unicode转换为纯ASCII以限制输入
将Unicode字符串替换为ASCII并只允许一小部分允许的字符 -->

In [3]:
# # 使用 “_” 表示非词汇表的字符与模型未处理字符
# allowed_characters = string.ascii_letters + string.digits + ".,;" + "_"
# n_letters = len(allowed_characters)

In [4]:
# # 删除Python unicode字符串中的重音符号
# # REF https://stackoverflow.com/a/518232/2809427
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#         and c in allowed_characters
#     )

<!-- #### 2. 文本 to 张量

使用 \<1 $\times$ n_letters\> 的 one-hot 编码，表示单个字符。

使用一个 2D 矩阵 \<line_length $\times$ 1 $\times$ n_letters\> 表示一个单词

额外的维度 1 是因为PyTorch假定所有内容都是批量的，这里使用批量大小为 1 -->

In [5]:
# # 字符索引定位
# def letterToIndex(letter):
#     return allowed_characters.find(letter in allowed_characters and letter or '_')

In [6]:
# # 词的张量表示
# def lineToTensor(line):
#     indices = torch.tensor([letterToIndex(letter=letter) for letter in line])
#     return F.one_hot(indices, num_classes=n_letters)

<!-- #### 3. 构建数据集

使用 Dataset 与 DataLoader 存储数据集，实现 `__init__`, `__len__`, `__getitem__` -->

In [7]:
# import re
# from pathlib import Path
# import pandas as pd
# import numpy as np
# import glob
# import os
# import time

# from torch.utils.data import Dataset

In [8]:
# data_path = Path('./data/Lexis-Nexis_LRA.csv')

In [9]:
# df_data = pd.read_csv(filepath_or_buffer=data_path)
# df_data.head()

In [10]:
# df_data.Text.__len__()

In [11]:
# df_data.drop_duplicates('Text').Text.apply(lambda text: bool(re.search(r'[^\x00-\x7F]',  text)) ^ True).sum()

In [12]:
# df_data.drop_duplicates('Text').loc[df_data.Text.apply(lambda text: bool(re.search(r'[^\x00-\x7F]',  text)) ^ True)]

In [13]:
# class sDataset(Dataset):
#     def __init__(self, df_data):
#         self.df = df_data.drop_duplicates('Text')
#         legal_indices = self.df.Text.apply(self.is_legal_string)
#         self.data = self.df.loc[legal_indices]
#         self.length = legal_indices.sum()

#     # 正确编码
#     def is_legal_string(self, text):
#         return bool(re.search(r'[^\x00-\x7F]',  text)) ^ True

#     def __len__(self):
#         return self.length

#     def __getitem__(self, index):
#         return self.data.iloc[index]


In [14]:
# sdata = sDataset(df_data=df_data)
# len(sdata), sdata[0]

In [15]:
# ASK_TEMPLATE = """
# - ASK: 
# ``` txt
# {}

# 请根据以下六个基本标准，对上文进行0-1标注

# 基本标准：
# (1)“PF_score”，报告提到伙伴部队的成功
# (2)“PF_US”，报告提到美国与伙伴部队合作
# (3)“PF_neg”，报告美国伙伴部队的负面情况
# (4)“Threat_up”，报告提到上帝抵抗军的威胁增加
# (5)“Threat_down”，上帝抵抗军的威胁是否减少
# (6)“Citizen_impact”，公民是否受到上帝抵抗军暴力的影响
# ```

# - RESPONSE:
# ```

# ```
# ---
# """

In [16]:
# size = 100
# with open('./data/MarkingByDeepSeek.md', 'w') as f:
#     f.write((ASK_TEMPLATE * size) .format(*sdata[:size].Text))

In [17]:
# with open('./data/MarkingByDeepSeek.md', 'r') as f:
#     Marking = f.read()

In [18]:
# def markingTodict(marking):
#     pattern = re.compile("""\s- ASK:\s\n```\stxt\n([\w\W]+?)\n\n请根据以下六个基本标准，对上文进行0-1标注\n\n基本标准：\n\(1\)“PF_score”，报告提到伙伴部队的成功\n\(2\)“PF_US”，报告提到美国与伙伴部队合作\n\(3\)“PF_neg”，报告美国伙伴部队的负面情况\n\(4\)“Threat_up”，报告提到上帝抵抗军的威胁增加\n\(5\)“Threat_down”，上帝抵抗军的威胁是否减少\n\(6\)“Citizen_impact”，公民是否受到上帝抵抗军暴力的影响\n```\n\n-\sRESPONSE:\n```\n([\w\W]+?)\n```\n""")
#     items = {}
#     for idx, (text, response) in enumerate(pattern.findall(marking)):
#         item = re.compile("([PFThreatCitizen]+_[\w]+).+?(\d)").findall(response)[:6]
#         items[idx] = {
#             'text': text,
#             'response': response,
#             **dict(item)
#         }
#     return items


In [19]:
# df_result = pd.DataFrame.from_dict(markingTodict(Marking), orient='index')
# df_result

In [20]:
# df_result.iloc[:, [0, 2, 3, 4, 5, 6, 7]].to_csv('./data/MarkingResult.csv', index=False)


In [21]:
# df_result = pd.read_csv('./data/MarkingResult.csv')
# df_result

In [22]:
import pandas as pd
from itertools import compress
import ast # REF https://docs.python.org/zh-cn/3.11/library/ast.html
import re


# 1. 整理数据


In [23]:
# 读取数据
data_df = pd.read_csv('./data/Lexis-Nexis_LRA.csv')
data_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",2012,March,19,['The International Criminal Court has reached...
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[""Geneva, Mar 31 (IBNS) The United Nations ref..."
2,'Now Katine is famous': how a Guardian project...,The Guardian(London),"December 30, 2017 Saturday 9:00 AM GMT",2017,December,30,"[""Joyce Abiro was among the first to join the ..."
3,Review: COVER STORY: KONY 2012: What happened ...,The Observer (London),15-Jul-12,2012,July,15,"['For 25 years, Joseph Kony and his Lord\'s Re..."
4,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",2011,February,15,['Displaced northern Ugandans have flocked hom...


In [24]:
data_df.shape

(14664, 7)

In [25]:
# 规范编码
def is_legal_text(text):
    if re.search(r'[^\x00-\x7F]',  text):
        return False
    try:
        ast.literal_eval(text)
        return True
    except:
        return False

In [26]:
data_df.Text.apply(is_legal_text).sum()

np.int64(12223)

In [27]:
data_df = data_df.loc[data_df.Text.apply(is_legal_text)]
data_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",2012,March,19,['The International Criminal Court has reached...
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[""Geneva, Mar 31 (IBNS) The United Nations ref..."
3,Review: COVER STORY: KONY 2012: What happened ...,The Observer (London),15-Jul-12,2012,July,15,"['For 25 years, Joseph Kony and his Lord\'s Re..."
4,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",2011,February,15,['Displaced northern Ugandans have flocked hom...
6,BRITISH SAFARI MAN ACCUSED OF MASS MURDER 'FRAMED,MAIL ON SUNDAY (London),"May 13, 2012 Sunday",2012,May,13,"[""LOCAL POACHERS'"", 'FROM TIMOTHY EVANS IN BAN..."


In [28]:
data_df.reset_index(drop=True, inplace=True)
data_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",2012,March,19,['The International Criminal Court has reached...
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[""Geneva, Mar 31 (IBNS) The United Nations ref..."
2,Review: COVER STORY: KONY 2012: What happened ...,The Observer (London),15-Jul-12,2012,July,15,"['For 25 years, Joseph Kony and his Lord\'s Re..."
3,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",2011,February,15,['Displaced northern Ugandans have flocked hom...
4,BRITISH SAFARI MAN ACCUSED OF MASS MURDER 'FRAMED,MAIL ON SUNDAY (London),"May 13, 2012 Sunday",2012,May,13,"[""LOCAL POACHERS'"", 'FROM TIMOTHY EVANS IN BAN..."


In [29]:
# # 保存去除非法编码后的数据
# data_df.to_csv('./data/Lexis-Nexis_LRA_cleaned_0.csv', index=False)

In [30]:
# # 将Unicode转换为纯ASCII以限制输入
# # 将Unicode字符串替换为ASCII并只允许一小部分允许的字符
# # 使用 “_” 表示非词汇表的字符与模型未处理字符
# allowed_characters = string.ascii_letters + string.digits + ".,;" + "_"
# n_letters = len(allowed_characters)
# n_letters

In [31]:
# # 删除Python unicode字符串中的重音符号
# # REF https://stackoverflow.com/a/518232/2809427
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#         and c in allowed_characters
#     )

## 1.1. 过滤可用的报告

<!-- 
(1) “PF_score”，报告提到伙伴部队的成功 - 0  
(2) “PF_US”，报告提到美国与伙伴部队合作 - 0  
(3) “PF_neg”，报告美国伙伴部队的负面情况 - 0
(4) “Threat_up”，报告提到上帝抵抗军的威胁增加 - 1  
(5) “Threat_down”，上帝抵抗军的威胁是否减少 - 0  
(6) “Citizen_impact”，公民是否受到上帝抵抗军暴力的影响 - 1  
 -->


In [32]:
data_df = pd.read_csv('./data/Lexis-Nexis_LRA_cleaned_0.csv')
data_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",2012,March,19,['The International Criminal Court has reached...
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[""Geneva, Mar 31 (IBNS) The United Nations ref..."
2,Review: COVER STORY: KONY 2012: What happened ...,The Observer (London),15-Jul-12,2012,July,15,"['For 25 years, Joseph Kony and his Lord\'s Re..."
3,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",2011,February,15,['Displaced northern Ugandans have flocked hom...
4,BRITISH SAFARI MAN ACCUSED OF MASS MURDER 'FRAMED,MAIL ON SUNDAY (London),"May 13, 2012 Sunday",2012,May,13,"[""LOCAL POACHERS'"", 'FROM TIMOTHY EVANS IN BAN..."


In [33]:
data_df.Text = data_df.Text.apply(ast.literal_eval)
data_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",2012,March,19,[The International Criminal Court has reached ...
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[Geneva, Mar 31 (IBNS) The United Nations refu..."
2,Review: COVER STORY: KONY 2012: What happened ...,The Observer (London),15-Jul-12,2012,July,15,"[For 25 years, Joseph Kony and his Lord's Resi..."
3,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",2011,February,15,[Displaced northern Ugandans have flocked home...
4,BRITISH SAFARI MAN ACCUSED OF MASS MURDER 'FRAMED,MAIL ON SUNDAY (London),"May 13, 2012 Sunday",2012,May,13,"[LOCAL POACHERS', FROM TIMOTHY EVANS IN BANGUI..."


In [34]:
def abstract_text(text, target):
    matches = list(
        map(
            lambda s: any(map(lambda t: t in s, target)),
            text
        )
    )
    return list(
        map(
            lambda _: any(_),
            zip(
                matches, [False] + matches[:-1], matches[1:] + [False]
            )
        )
    )

- 伙伴部队
<!-- 
乌干达人民国防军 - Uganda
苏丹人民解放军 - Sudan
中非武装部队 - Central African Republic, CAR
刚果民主共和国武装部队 - Democratic Republic of the Congo, DRC
 -->



In [35]:
PF_targets = [
    ("Uganda",),
    ("Sudan",),
    ("Central African Republic", "CAR"),
    ("Democratic Republic of the Congo", "DRC",),
]

In [36]:
PF_matched = data_df.Text.apply(lambda s: abstract_text(s, PF_targets[0])).apply(lambda x: any(x))
for PF_target in PF_targets[1:]:
    PF_matched |= data_df.Text.apply(lambda s: abstract_text(s, PF_target)).apply(lambda x: any(x))
PF_df = data_df.loc[PF_matched]
PF_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",2012,March,19,[The International Criminal Court has reached ...
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[Geneva, Mar 31 (IBNS) The United Nations refu..."
3,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",2011,February,15,[Displaced northern Ugandans have flocked home...
4,BRITISH SAFARI MAN ACCUSED OF MASS MURDER 'FRAMED,MAIL ON SUNDAY (London),"May 13, 2012 Sunday",2012,May,13,"[LOCAL POACHERS', FROM TIMOTHY EVANS IN BANGUI..."
5,National Security Sudan,CountryWatch Reviews,9-Dec-17,2017,December,0,"[External Threats, The Sudanese government is ..."


In [37]:
PF_df.shape

(11037, 7)

In [38]:
# PF_df.reset_index(drop=True, inplace=True)
# PF_df.to_csv('./data/PF_matched.csv', index=False)

- 上帝抵抗军
<!-- 
上帝抵抗军 - Lord's Resistance Army, LRA
 -->

In [39]:
LRA_targets = [
    ("Lord's Resistance Army", "LRA",),
]

In [40]:
LRA_matched = data_df.Text.apply(lambda s: abstract_text(s, LRA_targets[0])).apply(lambda x: any(x))
for LRA_target in LRA_targets[1:]:
    LRA_matched |= data_df.Text.apply(lambda s: abstract_text(s, LRA_target)).apply(lambda x: any(x))
LRA_df = data_df.loc[LRA_matched]
LRA_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,War criminal walking around in plain sight,The Calgary Herald (Alberta),"March 19, 2012 Monday",2012,March,19,[The International Criminal Court has reached ...
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[Geneva, Mar 31 (IBNS) The United Nations refu..."
2,Review: COVER STORY: KONY 2012: What happened ...,The Observer (London),15-Jul-12,2012,July,15,"[For 25 years, Joseph Kony and his Lord's Resi..."
3,Uganda 's war-scarred north set for peacetime ...,Agence France Presse -- English,"February 15, 2011 Tuesday 9:21 AM GMT",2011,February,15,[Displaced northern Ugandans have flocked home...
4,BRITISH SAFARI MAN ACCUSED OF MASS MURDER 'FRAMED,MAIL ON SUNDAY (London),"May 13, 2012 Sunday",2012,May,13,"[LOCAL POACHERS', FROM TIMOTHY EVANS IN BANGUI..."


In [41]:
LRA_df.shape

(11616, 7)

In [42]:
# LRA_df.reset_index(drop=True, inplace=True)
# LRA_df.to_csv('./data/LRA_matched.csv', index=False)

- 伙伴部队 + 上帝抵抗军

In [43]:
all_targets = [
    ("Uganda",),
    ("Sudan",),
    ("Central African Republic", "CAR"),
    ("Democratic Republic of the Congo", "DRC",),
] + [
    ("Lord's Resistance Army", "LRA",),
]

In [44]:
all_matched = data_df.Text.apply(lambda s: abstract_text(s, all_targets[0])).apply(lambda x: any(x))
for all_target in all_targets[1:]:
    all_matched &= data_df.Text.apply(lambda s: abstract_text(s, all_target)).apply(lambda x: any(x))
all_df = data_df.loc[all_matched]
all_df.head()

,Title,Source,Time,Year,Month,Day,Text
1,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[Geneva, Mar 31 (IBNS) The United Nations refu..."
10,Uganda U.S. Legislator Proposes Crackdown on R...,Africa News,"March 1, 2010 Monday",2010,March,0,[American legislators are pushing for a law th...
20,"The Wizard of the Nile, Matthew Green, Portobe...",The Point,1-Apr-16,2016,April,1,[Once again this week Uganda is in the news fo...
25,US Pushing for New Law Against LRA,The East African (Nairobi),"March 1, 2010 Monday",2010,March,0,[American legislators are pushing for a law th...
35,"AU, UN Take Stand Against Kony",The New Times (Kigali),"April 13, 2012 Friday",2012,April,13,[As Rwanda marks the 18th Anniversary of the 1...


In [45]:
all_df.shape

(1329, 7)

In [46]:
# all_df.reset_index(drop=True, inplace=True)
# all_df.to_csv('./data/all_matched.csv', index=False)

## 1.2. 删除不必要的重复


In [47]:
all_df = pd.read_csv('./data/all_matched.csv')
all_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[""Geneva, Mar 31 (IBNS) The United Nations ref..."
1,Uganda U.S. Legislator Proposes Crackdown on R...,Africa News,"March 1, 2010 Monday",2010,March,0,"[""American legislators are pushing for a law t..."
2,"The Wizard of the Nile, Matthew Green, Portobe...",The Point,1-Apr-16,2016,April,1,"[""Once again this week Uganda is in the news f..."
3,US Pushing for New Law Against LRA,The East African (Nairobi),"March 1, 2010 Monday",2010,March,0,"[""American legislators are pushing for a law t..."
4,"AU, UN Take Stand Against Kony",The New Times (Kigali),"April 13, 2012 Friday",2012,April,13,"[""As Rwanda marks the 18th Anniversary of the ..."


In [48]:
all_df.shape

(1329, 7)

In [49]:
diverse_df = all_df.loc[all_df.Text.duplicated() ^ True]
diverse_df.shape

(1157, 7)

In [50]:
# diverse_df.reset_index(drop=True, inplace=True)
# diverse_df.to_csv('./data/diverse_all_matched.csv', index=False)

## 1.3. 重新组织数据


In [51]:
diverse_df = pd.read_csv('./data/diverse_all_matched.csv')
diverse_df.head()

,Title,Source,Time,Year,Month,Day,Text
0,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[""Geneva, Mar 31 (IBNS) The United Nations ref..."
1,Uganda U.S. Legislator Proposes Crackdown on R...,Africa News,"March 1, 2010 Monday",2010,March,0,"[""American legislators are pushing for a law t..."
2,"The Wizard of the Nile, Matthew Green, Portobe...",The Point,1-Apr-16,2016,April,1,"[""Once again this week Uganda is in the news f..."
3,"AU, UN Take Stand Against Kony",The New Times (Kigali),"April 13, 2012 Friday",2012,April,13,"[""As Rwanda marks the 18th Anniversary of the ..."
4,"UN ,Governments to meet to finalize regional s...",IBNS,"March 14, 2012 Wednesday 6:30 AM EST",2012,March,14,"[""New York, Mar 14 (IBNS): United Nations and ..."


In [52]:
diverse_df.shape

(1157, 7)

In [53]:
diverse_df.Text = diverse_df.Text.apply(ast.literal_eval)

In [54]:
def abstract_text(text, target):
    matches = list(
        map(
            lambda s: any(map(lambda t: t in s, target)),
            text
        )
    )
    return list(
        map(
            lambda _: any(_),
            zip(
                matches, [False] + matches[:-1], matches[1:] + [False]
            )
        )
    )

In [55]:
PF_targets = [
    ("Uganda",),
    ("Sudan",),
    ("Central African Republic", "CAR"),
    ("Democratic Republic of the Congo", "DRC",),
]

In [56]:
PF_matched = diverse_df.Text.apply(lambda s: abstract_text(s, PF_targets[0]))
for PF_target in PF_targets[1:]:
    PF_matched = map(lambda x: list(map(any, zip(x[0], x[1]))), zip(PF_matched, diverse_df.Text.apply(lambda s: abstract_text(s, PF_target))))
diverse_df['PF_TARGET'] = list(PF_matched)
diverse_df.head()

,Title,Source,Time,Year,Month,Day,Text,PF_TARGET
0,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[Geneva, Mar 31 (IBNS) The United Nations refu...","[True, True, True, True, True, True, True, Tru..."
1,Uganda U.S. Legislator Proposes Crackdown on R...,Africa News,"March 1, 2010 Monday",2010,March,0,[American legislators are pushing for a law th...,"[True, True, True, True, True, True, True, Fal..."
2,"The Wizard of the Nile, Matthew Green, Portobe...",The Point,1-Apr-16,2016,April,1,[Once again this week Uganda is in the news fo...,"[True, True, True, True, True, True, True, False]"
3,"AU, UN Take Stand Against Kony",The New Times (Kigali),"April 13, 2012 Friday",2012,April,13,[As Rwanda marks the 18th Anniversary of the 1...,"[True, True, True, True, True, True, True, Tru..."
4,"UN ,Governments to meet to finalize regional s...",IBNS,"March 14, 2012 Wednesday 6:30 AM EST",2012,March,14,"[New York, Mar 14 (IBNS): United Nations and g...","[True, True, True, True, True, False, True, Tr..."


In [57]:
LRA_targets = [
    ("Lord's Resistance Army", "LRA",),
]

In [58]:
LRA_matched = diverse_df.Text.apply(lambda s: abstract_text(s, LRA_targets[0]))
for LRA_target in LRA_targets[1:]:
    LRA_matched = map(lambda x: list(map(any, zip(x[0], x[1]))), zip(LRA_matched, diverse_df.Text.apply(lambda s: abstract_text(s, LRA_target))))
diverse_df['LRA_TARGET'] = list(LRA_matched)
diverse_df.head()

,Title,Source,Time,Year,Month,Day,Text,PF_TARGET,LRA_TARGET
0,UN warns of growing attacks in Central Africa,IBNS,"March 31, 2012 Saturday 6:30 AM EST",2012,March,31,"[Geneva, Mar 31 (IBNS) The United Nations refu...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
1,Uganda U.S. Legislator Proposes Crackdown on R...,Africa News,"March 1, 2010 Monday",2010,March,0,[American legislators are pushing for a law th...,"[True, True, True, True, True, True, True, Fal...","[True, True, True, True, True, True, True, Fal..."
2,"The Wizard of the Nile, Matthew Green, Portobe...",The Point,1-Apr-16,2016,April,1,[Once again this week Uganda is in the news fo...,"[True, True, True, True, True, True, True, False]","[True, True, True, True, True, True, False, Fa..."
3,"AU, UN Take Stand Against Kony",The New Times (Kigali),"April 13, 2012 Friday",2012,April,13,[As Rwanda marks the 18th Anniversary of the 1...,"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
4,"UN ,Governments to meet to finalize regional s...",IBNS,"March 14, 2012 Wednesday 6:30 AM EST",2012,March,14,"[New York, Mar 14 (IBNS): United Nations and g...","[True, True, True, True, True, False, True, Tr...","[True, True, True, True, True, True, True, Tru..."


In [59]:
diverse_df.to_csv('./data/result_1.csv', index=False)

<!-- # 2. 开发编码标准 -->


<!-- # 3. 构建训练数据 -->


<!-- # 4. 校准机器学习算法 -->


<!-- # 5. 将ML算法的输出转换成评估就绪的格式 -->
